In [1]:
import numpy as np
import pandas as pd
import os
import sys

In [2]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)

In [3]:
res_folder = "./disagreement_matrix/engineering_version/auc_results/"
dis_folder = "./disagreement_matrix/engineering_version/file_disagreement_records/"


In [4]:
auc_results = dict()
for root, path, files in os.walk(res_folder):
    for file in files:
        if file.endswith("xlsx"):
            auc_results[file[:-5]] = pd.read_excel(root+file, index_col=0)

In [5]:
columns = auc_results["WPBC_withoutdupl_norm_auc_collection"].combination.values.tolist()
fnames = []
auc_records = []

for fname in auc_results:
    fname_prefix = fname[:-(len("auc_collection")+1)]
    auc_res = auc_results[fname].iloc[:,1].values
    auc_records.append(auc_res)
    fnames.append(fname_prefix)
    
target_df = pd.DataFrame(auc_records, index=fnames, columns=columns)

In [13]:
target_df.loc[:,["ensem_sum_self:thr_2std:dis_sum_rsd",'ensem_sum_self:thr_2std:dis_mad']].to_excel("tmp_result.xlsx")

In [6]:
# key as comb, value is fname
surpass_record = dict()
margin = 0.001
for fname in auc_results.keys():
    df_ = auc_results[fname]
    best_base_auc = df_.auc[df_.combination=="best_base_auc"].values[0]
    good_records = df_.combination[df_.auc>best_base_auc+margin]
    if len(good_records)>0:
        for m in good_records.values:
            if m in surpass_record.keys():
                surpass_record[m].add((fname, df_.auc[df_.combination==m].values[0] - best_base_auc))
            else:
                surpass_record[m] = set([(fname, df_.auc[df_.combination==m].values[0] - best_base_auc)])
                
                
res_list = []
for key, value in surpass_record.items():
    res_list.append((key, len(value), np.mean([x[1] for x in value])))

In [7]:
all_files_record = {"best_base_auc":[], "ensemble_best_auc":[],'best_comb':[]}
fnames = []
for fname in auc_results.keys():
    df_ = auc_results[fname]
    best_base_auc = df_.auc[df_.combination=="best_base_auc"].values[0]
    merge_best = np.max(df_.auc[df_.combination!="best_base_auc"].values)
    guess_best = df_.combination[df_.auc==merge_best].values[0]
    all_files_record['best_base_auc'].append(best_base_auc)
    all_files_record['ensemble_best_auc'].append(merge_best)
    all_files_record['best_comb'].append( guess_best)
    fnames.append(fname)
    
all_files_record['DataSets'] = fnames
df_res = pd.DataFrame.from_dict(all_files_record)

In [8]:

pd.DataFrame(sorted(res_list,key=lambda x: 
                    x[1],reverse=True),columns=['ensemble_methods',
                    'num_surpass','avg_better_auc']).sort_values(['num_surpass','avg_better_auc'],ascending=[False
                                                                                                        ,False]).to_excel("./LOF_base_detectors_ensemble.xlsx")

pd.DataFrame(sorted(res_list,key=lambda x: 
                    x[1],reverse=True),columns=['ensemble_methods',
                    'num_surpass','avg_better_auc']).sort_values(['num_surpass','avg_better_auc'],ascending=[False
                                                                                                        ,False])

,ensemble_methods,num_surpass,avg_better_auc
0,ensem_max:thr_2std:dis_std,6,0.022531
1,ensem_max:thr_2std:dis_mad,6,0.022531
2,ensem_max:thr_2std:dis_sum_rsd,6,0.022531
3,ensem_max:thr_2std:dis_std_rsd,6,0.022531
4,ensem_max:thr_2std:dis_max_rsd,6,0.022531
5,ensem_max:thr_iqr:dis_std,6,0.022531
6,ensem_max:thr_iqr:dis_mad,6,0.022531
7,ensem_max:thr_iqr:dis_sum_rsd,6,0.022531
8,ensem_max:thr_iqr:dis_std_rsd,6,0.022531
9,ensem_max:thr_iqr:dis_max_rsd,6,0.022531


In [9]:
df_res['best_base_auc'] = np.round(df_res['best_base_auc'],3)
df_res['ensemble_best_auc'] = np.round(df_res['ensemble_best_auc'],3)
df_res['diff'] = df_res['ensemble_best_auc'] - df_res['best_base_auc']
q = df_res.loc[:,['ensemble_best_auc','best_base_auc','diff','DataSets','best_comb']]
q.sort_values('diff', ascending=False)

,ensemble_best_auc,best_base_auc,diff,DataSets,best_comb
30,0.935,0.853,0.082,Lymphography_withoutdupl_catremoved_auc_collec...,ensem_mean:thr_2std:dis_std
18,0.750,0.713,0.037,InternetAds_withoutdupl_norm_19_auc_collection,ensem_sum_self:thr_2std:dis_std
13,0.781,0.748,0.033,Waveform_withoutdupl_v01_auc_collection,ensem_sum_self:thr_2std:dis_std
20,0.963,0.946,0.017,Shuttle_withoutdupl_v01_auc_collection,ensem_sido:thr_2std:dis_std
27,0.882,0.866,0.016,Glass_withoutdupl_norm_auc_collection,ensem_max:thr_2std:dis_std
24,0.996,0.994,0.002,satimage-2_auc_collection,ensem_sido:thr_2std:dis_std
36,0.763,0.761,0.002,Arrhythmia_withoutdupl_46_auc_collection,ensem_sodi:thr_2std:dis_std
7,0.990,0.990,0.000,WBC_v01_auc_collection,ensem_sodi:thr_2std:dis_std
21,0.998,0.998,0.000,wine_auc_collection,ensem_sido:thr_2std:dis_std
4,0.989,0.992,-0.003,KDDCup99_withoutdupl_catremoved_auc_collection,ensem_sodi:thr_2std:dis_std


In [ ]:
q.to_excel("./LOF_base_overall_perform.xlsx")

In [ ]:
weights_dfname = dict()
for file in os.listdir("/Users/kadima/experiment_any/anomaly-detection/disagreement_matrix/engineering_version/rsj_matrix_records/"):
    if file.endswith("xlsx"):
        weights_dfname[file[:-5]] = pd.read_excel("/Users/kadima/experiment_any/anomaly-detection/disagreement_matrix/engineering_version/rsj_matrix_records/"+file, index_col=0,header=0)

In [ ]:
dis_dfname = dict()
for file in os.listdir(dis_folder):
    if file.endswith("xlsx"):
        dis_dfname[file[:-5]] = pd.read_excel(dis_folder+file, index_col=0,header=0)

In [ ]:
weights_dfname.keys()

In [ ]:
weights_dfname['optdigits_odi']#.mean()

In [ ]:
dis_dfname["optdigits_threshold"]

In [ ]:
dis_dfname["optdigits_threshold"].value_counts()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# unchanged = weights_dfname['breastw_auc_collection_self_sum'].loc[dis_dfname["optdigits_threshold"].values,:]
changed = weights_dfname['InternetAds_withoutdupl_norm_19_self_sum']#.loc[~dis_dfname["optdigits_threshold"].values,:]

In [ ]:
changed.shape

In [ ]:
unchanged.shape

In [ ]:
x= np.arange(1,8)
for j in range(1500):
#     plt.plot(x, unchanged.iloc[j,:])
    plt.scatter(x, changed.iloc[j,:],c='gray',alpha=0.05)
plt.show()

In [ ]:
x= np.arange(1,13)
for j in range(600):
    plt.plot(x, changed.iloc[j,:])
plt.show()

In [ ]:
weights_dfname['optdigits_odi']